Test Loss: 1.0770130157470703
Test Accuracy: 0.6356924176216125

Epoch 30/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 45s 100ms/step - accuracy: 0.2452 - loss: 2.4779 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2492 - val_loss: 2.4545 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05

filename: efficientnet_b3_60_l_67%.h5

In [ ]:
# imports and some initial setup
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

emotion_map = {
  0: 'Angry',
  1: 'Disgust',
  2: 'Fear',
  3: 'Happy',
  4: 'Sad',
  5: 'Surprise',
  6: 'Neutral'
}

train_data_raw = pd.read_csv('train.csv')
test_data_raw = pd.read_csv('test.csv')

# create a data generator to not load all images into memory
def data_generator(data, batch_size=32):
  while True:
    for start in range(0, len(data), batch_size):
      end = min(start + batch_size, len(data))
      batch_data = data[start:end]
      batch_features = []
      batch_labels = []
      for _, row in batch_data.iterrows():
        img = np.array(row['pixels'].split(), 'float32').reshape(48, 48)
        img /= 255.0
        img_expanded = np.expand_dims(img, axis=-1)
        img_rgb = np.repeat(img_expanded, 3, axis=-1)
        batch_features.append(img_rgb)
        batch_labels.append(row['emotion'])
      yield np.stack(batch_features), to_categorical(batch_labels, num_classes=len(emotion_map))

train_generator = data_generator(train_data_raw)

In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


max_epochs_per_stage = 30
BATCH_SIZE = 64

# Load the EfficientNetB5 model pre-trained on ImageNet, excluding the top layers
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Add custom top layers with higher dropout rates
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)  # Increased dropout
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.6)(x)  # Increased dropout
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.6)(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x) 
predictions = Dense(len(emotion_map), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers initially
for layer in base_model.layers[-60:]:
  layer.trainable = True

# Compile with initial settings
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Create an ImageDataGenerator for data augmentation suited for facial expression data
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to augment data
def augmented_data_generator(generator, datagen, batch_size=32):
    while True:
        batch_features, batch_labels = next(generator)
        augmented_data = datagen.flow(batch_features, batch_labels, batch_size=batch_size, shuffle=False)
        yield next(augmented_data)

train_generator_augmented = augmented_data_generator(data_generator(train_data_raw), datagen, BATCH_SIZE)


model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

history = model.fit(
    train_generator_augmented,
    steps_per_epoch=len(train_data_raw) // BATCH_SIZE,
    epochs=max_epochs_per_stage,
    callbacks=[early_stopping, reduce_lr],
    validation_data=augmented_data_generator(data_generator(test_data_raw), datagen),
    validation_steps=len(test_data_raw) // BATCH_SIZE,
)


# Save the final model after modular unfreezing
model.save('efficientnetb5_finetuned_model.h5')
print("Model fine-tuning complete and saved as 'efficientnetb5_finetuned_model_2.h5'.")


In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.utils import to_categorical

# Re-create test features and labels with consistent preprocessing
test_features = []
test_labels = []
for _, row in test_data_raw.iterrows():
    img = np.array(row['pixels'].split(), 'float32').reshape(48, 48)
    img /= 255.0  # normalize
    img_expanded = np.expand_dims(img, axis=-1)
    img_rgb = np.repeat(img_expanded, 3, axis=-1)
    test_features.append(img_rgb)
    test_labels.append(row['emotion'])

test_features = np.array(test_features)
test_labels_categorical = to_categorical(test_labels, num_classes=len(emotion_map))

# Evaluate the model
evaluation = model.evaluate(test_features, test_labels_categorical, batch_size=BATCH_SIZE)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")